In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import scipy.special as ssp

import pyro
from pyro.distributions import Normal, Bernoulli
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
# for CI testing
smoke_test = ('CI' in os.environ)
pyro.enable_validation(True)

In [2]:
def build_linear_dataset(N, p=1, noise_std=0.01):
    X = np.random.rand(N, p)
    # w = 3
    w = 3 * np.ones(p)
    # b = 1
    y = np.matmul(X, w) + np.repeat(1, N) + np.random.normal(0, noise_std, size=N)
    y = y.reshape(N, 1)
    X, y = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
    data = torch.cat((X, y), 1)
    assert data.shape == (N, p + 1)
    return data

In [3]:
def build_logistic_dataset(N, p=1, noise_std=0.01):
    X = np.random.randn(N, p)
    
    w = np.random.randn(p)
    w += 2 * np.sign(w)

    y = np.round(ssp.expit(np.matmul(X, w) 
                           + np.repeat(1, N) 
                           + np.random.normal(0, noise_std, size=N)))
    y = y.reshape(N, 1)
    X, y = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
    data = torch.cat((X, y), 1)
    assert data.shape == (N, p + 1)
    return data

In [5]:
class RegressionModel(nn.Module):
    def __init__(self, p):
        # p = number of features
        super(RegressionModel, self).__init__()
        self.linear = nn.Linear(p, 1)
        self.non_linear = nn.Sigmoid()

    def forward(self, x):
        return self.non_linear(self.linear(x))

In [6]:
def model(data):
    # Create unit normal priors over the parameters
    loc, scale = torch.zeros(1, p), 10 * torch.ones(1, p)
    bias_loc, bias_scale = torch.zeros(1), 10 * torch.ones(1)
    w_prior = Normal(loc, scale).independent(1)
    b_prior = Normal(bias_loc, bias_scale).independent(1)
    priors = {'linear.weight': w_prior, 'linear.bias': b_prior}
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", regression_model, priors)
    # sample a regressor (which also samples w and b)
    lifted_reg_model = lifted_module()
    with pyro.iarange("map", N):
        x_data = data[:, :-1]
        y_data = data[:, -1]
        
        model_logits = lifted_reg_model(x_data).squeeze(-1)
        pyro.sample("obs", Bernoulli(logits=model_logits, validate_args=True), obs=y_data.squeeze())

In [7]:
softplus = torch.nn.Softplus()

def guide(data):
    # define our variational parameters
    w_loc = torch.randn(1, p)
    # note that we initialize our scales to be pretty narrow
    w_log_sig = torch.tensor(-3.0 * torch.ones(1, p) + 0.05 * torch.randn(1, p))
    b_loc = torch.randn(1)
    b_log_sig = torch.tensor(-3.0 * torch.ones(1) + 0.05 * torch.randn(1))
    # register learnable params in the param store
    mw_param = pyro.param("guide_mean_weight", w_loc)
    sw_param = softplus(pyro.param("guide_log_scale_weight", w_log_sig))
    mb_param = pyro.param("guide_mean_bias", b_loc)
    sb_param = softplus(pyro.param("guide_log_scale_bias", b_log_sig))
    # guide distributions for w and b
    w_dist = Normal(mw_param, sw_param).independent(1)
    b_dist = Normal(mb_param, sb_param).independent(1)
    dists = {'linear.weight': w_dist, 'linear.bias': b_dist}
    # overload the parameters in the module with random samples
    # from the guide distributions
    lifted_module = pyro.random_module("module", regression_model, dists)
    # sample a regressor (which also samples w and b)
    return lifted_module()

In [10]:
N, p = 500, 100
optim = Adam({"lr": 0.05})
svi = SVI(model, guide, optim, loss=Trace_ELBO())
regression_model = RegressionModel(p)

In [89]:
pyro.clear_param_store()
data = build_logistic_dataset(N, p)
num_iterations = 10000
for j in range(num_iterations):
    # calculate the loss and take a gradient step
    loss = svi.step(data)
    if j % (num_iterations / 10) == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / float(N)))

[iteration 0001] loss: 1.6895
[iteration 1001] loss: 0.6626
[iteration 2001] loss: 0.6781
[iteration 3001] loss: 0.6591
[iteration 4001] loss: 0.6731
[iteration 5001] loss: 0.6768
[iteration 6001] loss: 0.6751
[iteration 7001] loss: 0.6663
[iteration 8001] loss: 0.6687
[iteration 9001] loss: 0.6735


In [90]:
for name in pyro.get_param_store().get_all_param_names():
    #print("[%s]: %.3f" % (name, pyro.param(name).data.numpy()))
    print("[%s]: %s" % (name, pyro.param(name).data.numpy()))

[guide_mean_weight]: [[ 10.121038    -2.7254305   -0.13062297  10.248975     7.4295087
   -3.6112792    7.7594714   10.111515     3.9126155    6.528598
    9.058292    -6.626209    -8.619249     1.796047    -5.913176
    0.6211253   12.51668      4.7814918   10.83625      7.074222
    4.394132    10.678343    -9.629215     0.01682232  -7.6484423
   -4.097059    11.537101    -4.7015104   -9.161883    -8.405314
   -7.876971    -8.807516     3.6903944    5.4311028    4.973605
  -13.16374     -5.419484     3.9035625    6.977791     5.290474
    1.9047701    3.9085135    7.427295    -3.4100535  -12.355433
    2.948364    -5.2586455    6.7773366   -9.540961     7.767003
    7.0676947    4.4470787    8.159762    -5.3476276   -7.0261817
    1.0553516    8.92021     -3.1246934    6.282368     4.339715
   -0.60011244  -2.5544064   -8.166295     1.5811864    6.28793
   -8.263433    -0.65276      3.1239996   -3.9836624    2.9059925
   -2.2632983    9.797131    -5.4767213    1.2379887    4.2444143


In [93]:
scorecard = 0
trainingdata = data
for index, prediction in enumerate(torch.round(sampled_reg_model(data[:,:-1]))):
    if prediction.item() == data[index,-1].item():
        scorecard += 1
print('Final Score: %s / %s or %s' %(scorecard, N, round(scorecard / N,3)))

Final Score: 284 / 500 or 0.568
